In [1]:
import json
from urllib import request, parse
from pprint import pprint

prompt_text = """
{
  "3": {
    "inputs": {
      "seed": 8566257,
      "steps": 20,
      "cfg": 7,
      "sampler_name": "euler",
      "scheduler": "normal",
      "denoise": 1,
      "model": [
        "4",
        0
      ],
      "positive": [
        "6",
        0
      ],
      "negative": [
        "7",
        0
      ],
      "latent_image": [
        "5",
        0
      ]
    },
    "class_type": "KSampler",
    "_meta": {
      "title": "KSampler"
    }
  },
  "4": {
    "inputs": {
      "ckpt_name": "SD15\\\\28DSTABLEBESTVERSION_v6.safetensors"
    },
    "class_type": "CheckpointLoaderSimple",
    "_meta": {
      "title": "Load Checkpoint"
    }
  },
  "5": {
    "inputs": {
      "width": 512,
      "height": 512,
      "batch_size": 1
    },
    "class_type": "EmptyLatentImage",
    "_meta": {
      "title": "Empty Latent Image"
    }
  },
  "6": {
    "inputs": {
      "text": "masterpiece best quality girl",
      "clip": [
        "4",
        1
      ]
    },
    "class_type": "CLIPTextEncode",
    "_meta": {
      "title": "CLIP Text Encode (Prompt)"
    }
  },
  "7": {
    "inputs": {
      "text": "bad hands",
      "clip": [
        "4",
        1
      ]
    },
    "class_type": "CLIPTextEncode",
    "_meta": {
      "title": "CLIP Text Encode (Prompt)"
    }
  },
  "8": {
    "inputs": {
      "samples": [
        "3",
        0
      ],
      "vae": [
        "4",
        2
      ]
    },
    "class_type": "VAEDecode",
    "_meta": {
      "title": "VAE Decode"
    }
  },
  "9": {
    "inputs": {
      "filename_prefix": "ComfyUI",
      "images": [
        "8",
        0
      ]
    },
    "class_type": "SaveImage",
    "_meta": {
      "title": "Save Image"
    }
  }
}
"""

def queue_prompt(prompt):
    p = {"prompt": prompt}
    data = json.dumps(p).encode('utf-8')
    req =  request.Request("http://127.0.0.1:8188/prompt", data=data)
    # Pretty print the response
    pprint(json.loads(request.urlopen(req).read()))



prompt = json.loads(prompt_text)
#set the text prompt for our positive CLIPTextEncode
prompt["6"]["inputs"]["text"] = "masterpiece best quality"

#set the seed for our KSampler node
prompt["3"]["inputs"]["seed"] = 5


queue_prompt(prompt)


{'node_errors': {},
 'number': 8,
 'prompt_id': '7b7adfd0-d7ae-432f-8a54-2fad3b539ffe'}


In [3]:
from colorama import Fore, Style

def print_colors(json_dict):
    # Recursive function to find the positive and negative prompt node numbers
    def find_prompts(data, class_type='KSampler'):
        for key, value in data.items():
            if isinstance(value, dict):
                if value.get('class_type') == class_type:
                    return value['inputs'].get('positive', [''])[0], value['inputs'].get('negative', [''])[0]
                positive_key, negative_key = find_prompts(value, class_type)
                if positive_key or negative_key:
                    return positive_key, negative_key
        return None, None

    # Recursive print function handling the traversal path for prompt keys
    def recursive_print(data, positive_key, negative_key, path=""):
        for key, value in data.items():
            json_traverse_path = f"{path}/{key}"  # Maintain the path traveled in the JSON structure
            if isinstance(value, dict):
                recursive_print(value, positive_key, negative_key, json_traverse_path)
            elif isinstance(value, list) and all(isinstance(item, dict) for item in value):
                for item in value:
                    recursive_print(item, positive_key, negative_key, json_traverse_path)
            else:
                # Helper function to print colored text based on the key
                if key == 'text':
                    if positive_key and positive_key in json_traverse_path:
                        print(Fore.GREEN + f"{key}: {Fore.LIGHTGREEN_EX}{value}" + Style.RESET_ALL)
                    elif negative_key and negative_key in json_traverse_path:
                        print(Fore.RED + f"{key}: {Fore.LIGHTRED_EX}{value}" + Style.RESET_ALL)
                elif key in ['sampler', 'scheduler', 'denoising_strength', 'steps', 'seed', 'cfg_scale', 'denoise', 'sampler_name', 'cfg']:
                    print(Fore.MAGENTA + f"{key}: {Fore.LIGHTMAGENTA_EX}{value}" + Style.RESET_ALL)
                elif key in ['init_images', 'image']:
                    print(Fore.YELLOW + f"{key}: {Fore.LIGHTYELLOW_EX}{value}" + Style.RESET_ALL)
    
    positive_node, negative_node = find_prompts(json_dict)
    if positive_node is not None and negative_node is not None:
        recursive_print(json_dict, positive_node, negative_node)
    else:
        print("No KSampler class with positive and negative prompts found.")

# Your dictionary as a variable
example_dict = {
    'prompt': {'3': {'inputs': {'seed': 1433870208, 'steps': 15, 'cfg': 7.0, 'sampler_name': 'euler', 'scheduler': 'normal', 'denoise': 0.1, 'model': ['4', 0], 'positive': ['6', 0], 'negative': ['7', 0], 'latent_image': ['10', 0]}, 'class_type': 'KSampler', '_meta': {'title': 'KSampler'}}, '4': {'inputs': {'ckpt_name': 'v1-5-pruned-emaonly.safetensors'}, 'class_type': 'CheckpointLoaderSimple', '_meta': {'title': 'Load Checkpoint'}}, '6': {'inputs': {'text': 'a red cube and a green sphere', 'clip': ['4', 1]}, 'class_type': 'CLIPTextEncode', '_meta': {'title': 'CLIP Text Encode (Prompt)'}}, '7': {'inputs': {'text': 'ugly, bad art, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, blurry, bad anatomy, blurred, watermark, grainy, tiling, signature, cut off, draft', 'clip': ['4', 1]}, 'class_type': 'CLIPTextEncode', '_meta': {'title': 'CLIP Text Encode (Prompt)'}}, '8': {'inputs': {'samples': ['3', 0], 'vae': ['4', 2]}, 'class_type': 'VAEDecode', '_meta': {'title': 'VAE Decode'}}, '9': {'inputs': {'filename_prefix': 'ComfyUI', 'images': ['8', 0]}, 'class_type': 'SaveImage', '_meta': {'title': 'Save Image'}}, '10': {'inputs': {'pixels': ['12', 0], 'vae': ['11', 0]}, 'class_type': 'VAEEncode', '_meta': {'title': 'VAE Encode'}}, '11': {'inputs': {'vae_name': 'vae-ft-mse-840000-ema-pruned.safetensors'}, 'class_type': 'VAELoader', '_meta': {'title': 'Load VAE'}}, '12': {'inputs': {'image': 'test/ai-render-1712331840-a red cube and a green sphere-1-before-m7xjn3pd.png', 'upload': 'image'}, 'class_type': 'LoadImage', '_meta': {'title': 'Load Image'}}}
}

# Call the new print_colors function with your dictionary
print_colors(example_dict)

seed: 1433870208
steps: 15
cfg: 7.0
sampler_name: euler
scheduler: normal
denoise: 0.1
text: a red cube and a green sphere
text: ugly, bad art, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, extra limbs, disfigured, deformed, body out of frame, blurry, bad anatomy, blurred, watermark, grainy, tiling, signature, cut off, draft
image: test/ai-render-1712331840-a red cube and a green sphere-1-before-m7xjn3pd.png
